In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)

/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym

2.0.0-alpha0
2.2.4-tf


/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWarning: Passing (type

In [2]:
import os
import csv
import time
import datetime
import random
import json
from collections import Counter
from math import sqrt
import gensim
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input,Conv2D,MaxPool2D,concatenate,Flatten,Dense,Dropout,Embedding,Reshape
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from bs4 import BeautifulSoup
import logging
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import multiprocessing
import yaml
import jieba

In [3]:
class Config(object):
    #数据集路径
    dataSource = "../marriage.txt"
    stopWordSource = "../stopword.txt"
    #分词后保留大于等于最低词频的词
    miniFreq=0
    #统一输入文本序列的定长，取了所有序列长度的均值。超出将被截断，不足则补0
    sequenceLength = 30 
    batchSize=128
    epochs=20
    numClasses = 5
    #训练集的比例
    rate = 0.8  
    #生成嵌入词向量的维度
    embeddingSize = 200
    #卷积核数
    numFilters = 128
    #卷积核大小
    filterSizes = [1,2,3,4,5]
    dropoutKeepProb = 0.5
    #L2正则系数
    l2RegLambda = 0.01
# 实例化配置参数对象
config = Config()

In [4]:
#中文语料
#设置输出日志
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

file = open("../marriage.txt") 
sentences=[]
with open('../marriage.txt') as fr:
    for line in fr.readlines():
        temp=line.strip().split('\t')
#         sentences.append(jieba.lcut(temp[0]))
        sentences.append(list(temp[0]))

model = word2vec.Word2Vec(sentences,size=config.embeddingSize,
                     min_count=config.miniFreq,
                     window=10,
                     workers=multiprocessing.cpu_count(),sg=1,
                     iter=20)
model.save('../word2VecModel')

In [5]:
model = gensim.models.Word2Vec.load('../word2VecModel')
model.wv.vocab.keys()

dict_keys(['嗯', '有', '啊', '反', '了', '看', '没', '那', '太', '好', '几', '年', '零', '四', '是', '的', '我', '现', '在', '边', '上', '客', '户', '哎', '你', '稍', '等', '一', '下', '先', '过', '挂', '掉', '这', '还', '办', '最', '久', '都', '两', '班', '可', '能', '次', '再', '完', '跟', '他', '见', '多', '认', '证', '就', '问', '题', '但', '三', '报', '考', '交', '离', '婚', '拿', '事', '时', '候', '直', '接', '今', '天', '晚', '早', '走', '对', '吧', '大', '米', '然', '后', '弄', '回', '来', '说', '让', '钱', '吗', '号', '厅', '七', '呀', '听', '很', '吵', '份', '整', '个', '拜', '哇', '倒', '投', '诉', '不', '单', '身', '陈', '倩', '豆', '瓣', '带', '放', '啦', '给', '留', '线', '人', '方', '便', '答', '哈', '丢', '八', '前', '哦', '换', '正', '版', '呢', '半', '怪', '家', '里', '面', '出', '全', '呃', '妹', '找', '司', '法', '着', '急', '电', '话', '行', '选', '咋', '肯', '定', '改', '搬', '总', '共', '念', '奇', '意', '思', '已', '经', '退', '进', '确', '到', '们', '协', '议', '书', '章', '明', '点', '每', '坐', '车', '把', '错', '包', '括', '强', '调', '喂', '去', '也', '干', '啥', '帮', '吓', '跳', '常', '声', '开', '相', '爱', '午', '打', '加', '油', '会', '玩', '咱',

In [6]:
# 数据预处理的类，生成训练集和测试集
class Dataset(object):
    def __init__(self, config):
        self.dataSource = config.dataSource
        self.stopWordSource = config.stopWordSource  
        # 每条输入的序列处理为定长
        self.sequenceLength = config.sequenceLength  
        self.embeddingSize = config.embeddingSize
        self.rate = config.rate
        self.miniFreq=config.miniFreq
        self.stopWordDict = {}
        self.trainReviews = []
        self.trainLabels = []
        self.evalReviews = []
        self.evalLabels = []
        self.wordEmbedding =None
        self.n_symbols=0
        self.wordToIndex = {}
        self.indexToWord = {}
        
    def readData(self, filePath):
        text=[]
        label=[]
        with open(filePath) as fr:
            for line in file:
                temp=line.strip().split('\t')
                text.append(temp[0])
                label.append(temp[1])
        print('data:',len(text),len(label))
#         texts = [jieba.cut(document) for document in text]
        texts = [list(document) for document in text]
        return texts, label
    
    def readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
    
    def getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        #中文
        model = gensim.models.Word2Vec.load('../word2VecModel')
        vocab = []
        wordEmbedding = []
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        wordEmbedding.append(np.zeros(self.embeddingSize))
        vocab.append("UNK")
        wordEmbedding.append(np.random.randn(self.embeddingSize))
        for word in words:
            vector =model[word]
            vocab.append(word)
            wordEmbedding.append(vector)           
        return vocab, np.array(wordEmbedding)
    
    def genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        allWords = [word for review in reviews for word in review]
        #去掉停用词
#         subWords = [word for word in allWords if word not in self.stopWordDict]
        #统计词频，排序
#         wordCount = Counter(subWords)
        wordCount = Counter(allWords)
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        #去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= self.miniFreq ]
        #获取词列表和顺序对应的预训练权重矩阵
        vocab, wordEmbedding = self.getWordEmbedding(words)
        
        self.wordEmbedding = wordEmbedding
        
        self.wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self.indexToWord = dict(zip(list(range(len(vocab))), vocab))
        self.n_symbols = len(self.wordToIndex) + 1
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self.wordToIndex, f)
        with open("../wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self.indexToWord, f)
            
    def reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论里面的词，根据词表，映射为index表示
        每条评论 用index组成的定长数组来表示
        """
        review=list(review)
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]
        return reviewVec

    def genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        reviews = []
        labels = []
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self.reviewProcess(x[i], self.sequenceLength, self.wordToIndex)
            reviews.append(reviewVec)
            labels.append([y[i]])    
        trainIndex = int(len(x) * rate)
        
        #trainReviews = sequence.pad_sequences(reviews[:trainIndex], maxlen=self.sequenceLength)
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        #evalReviews = sequence.pad_sequences(reviews[trainIndex:], maxlen=self.sequenceLength)
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")
        return trainReviews, trainLabels, evalReviews, evalLabels
         
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        #读取停用词
        self.readStopWord(self.stopWordSource)
        #读取数据集
        reviews, labels = self.readData(self.dataSource)
        #分词、去停用词
        #生成 词汇-索引 映射表和预训练权重矩阵，并保存
        self.genVocabulary(reviews)
        #初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self.genTrainEvalData(reviews, labels, self.rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels

data = Dataset(config)
data.dataGen()

data: 6321 6321


/Library/Python/3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [10]:
class IDCNNCRF2():
    def __init__(self,
                 vocab_size: int,  # 词的数量(词表的大小)
                 n_class: int,  # 分类的类别(本demo中包括小类别定义了7个类别)
                 max_len: int = 100,  # 最长的句子最长长度
                 embedding_dim: int = 128,  # 词向量编码长度
                 drop_rate: float = 0.5,  # dropout比例
                 ):
        self.vocab_size = vocab_size
        self.n_class = n_class
        self.max_len = max_len
        self.embedding_dim = embedding_dim
        self.drop_rate = drop_rate

    def create_model(self):
        """
        本网络的机构采用的是，
           Embedding
           直接进行2个常规一维卷积操作
           接上一个空洞卷积操作
           连接2个全连接层
           最后连接CRF层

        kernel_size 采用2、3、4

        cnn  特征层数: 256、256、512

        """
        inputs = Input(shape=(self.max_len,))
        x = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim)(inputs)
        x = Conv1D(filters=256,
                   kernel_size=2,
                   activation='relu',
                   padding='same',
                   dilation_rate=1)(x)
        x = Conv1D(filters=256,
                   kernel_size=3,
                   activation='relu',
                   padding='same',
                   dilation_rate=1)(x)
        x = Conv1D(filters=512,
                   kernel_size=4,
                   activation='relu',
                   padding='same',
                   dilation_rate=2)(x)
        x = Dropout(self.drop_rate)(x)
        x = Dense(1024)(x)
        x = Dropout(self.drop_rate)(x)
        x = Dense(self.n_class)(x)
        self.model = Model(inputs=inputs, outputs=x)
        self.compile()
        return self.model

    def compile(self):
        model.compile(optimizer=optimizers.Adam(lr=1e-3),
                 loss=losses.SparseCategoricalCrossentropy(),
                 metrics=['accuracy'])


In [9]:
wordEmbedding = data.wordEmbedding
n_symbols=data.n_symbols
model = IDCNNCRF2(n_symbols,wordEmbedding)
model.summary()

AttributeError: 'IDCNNCRF2' object has no attribute 'summary'